# 1. MAF Source Data Pre-Processing 
##   Filtering and Formatting

Version Updated Dec 11, Ready for publish

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))
from __future__ import print_function
import numpy as np
import pandas as pd
from pandas import DataFrame   
from pandas import pivot
from pandas import tools
import matplotlib.pyplot as plt
from sklearn import preprocessing
import os
from os.path import join, dirname
from tqdm import tqdm_notebook
import cv2
import re
import io
import glob
import random
import warnings
import pickle
import cloudpickle
import sys 
import azureml
import string
from scipy import stats
import pip
import csv
import pyparsing
import tempfile  
import logging
from IPython.display import SVG
import mkl
import graphviz
from azure.storage.blob import BlockBlobService
import datetime

In [2]:
os.chdir('D://MAFData')
base_dir = 'D://MAFData'
WorkingDir = os.path.join(base_dir, 'Workingdir')

os.getcwd()

'D:\\MAFData'

## Import Portfolio Source Data

In [3]:
blob_account_name = "<youraccountnamehere>" # fill in your blob account name
blob_account_key = "<youraccountkeyhere>"  # fill in your blob account key

In [4]:
mycontainer = "<yourcontainernamehere>"       # fill in the container name
myblobname = "<yourfile.whateverextension>"        # fill in the blob name.  in our case we had .rpt sql report files.
mydatafile = "Portfolio"        # fill in the output file name

In [8]:
#Read rpt from blob
mydata = pd.read_csv(Portfolio, sep='\t',error_bad_lines=False, index_col=False, dtype='unicode')
print(mydata.shape)

In [6]:
#Read in from rpt file from local drive for iterative work
# mydata = pd.read_csv('df_Portfolio.rpt', sep='\t',error_bad_lines=False, index_col=False, dtype='unicode')
# mydata.shape
# mydata.head(2)

In [7]:
#Account number is anonymized
mydata.shape
mydata.head(1)

(55978620, 45)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,BLOCK_1,BLOCK_2,ACCOUNT_STATUS,TRANSFER_DATE,ACTUAL_CURRENT_BALANCE,AIF_FLAG,...,Next_Statement_Date,No_Of_Due_Stmt,Pct_Code,Previous_Card_Number,Product,Written_Off_Flag,APR_Rate,Product_Fee_Type,Product_Category,Account_Block_Desc
0,MAFF_ACCT_0000000001,2016-11-27,OPEN,2016-11-27,NaN,NaN,D,NaN,0,Y,...,2016-12-03,0,004,0000000000000000,004,N,LOW,NON FEE,CASHBACK,OTHERS


In [8]:
#Filter to just the columns of interest

Portfolio = mydata[['Account_Number',
 'MISDATE',
 'ATTRITION_FLAG',
 'ACCOUNT_CREATION_DATE',
 'ACTUAL_CURRENT_BALANCE',
 'Credit_Limit',
 'LAST_ACTIVITY_DATE',
 'Last_Statement_Closing_Balance',
 'Last_Statement_Date',
 'Last_Statement_Minimum_Due',
 'Monthly_Salary',
 'Months_On_Book',
 'Next_Statement_Date',
 'Monthly_Salary',
 'Nationality_Group',
 'Next_Statement_Date',
 'No_Of_Due_Stmt',
 'Product',
 'APR_Rate',
 'Product_Fee_Type',
 'Product_Category']]

del mydata

In [17]:
#See revised dimensions
Portfolio.shape

(55978620, 21)

In [11]:
#Reformat date, filter and review head and tail of dataframe.
Portfolio['LAST_ACTIVITY_DATE'] = Portfolio['LAST_ACTIVITY_DATE'].astype('datetime64[ns]')
Portfolio = Portfolio.sort_values(['Account_Number', 'MISDATE'], ascending=[True, True])
Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape
Portfolio.head(2)
Portfolio.tail(1)
list(Portfolio.columns.values)

(55978620, 21)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,(55978619 rows affected),NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MAFF_ACCT_0000000001,2016-11-27,OPEN,2016-11-27,0,10000,NaT,0,2016-11-03,0,...,0,2016-12-03,10000,06. OTHERS,2016-12-03,0,004,LOW,NON FEE,CASHBACK


,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
55978619,MAFF_ACCT_0000269020,2017-09-30,OPEN,2017-09-30,0,5000,NaT,0,2017-09-05,0,...,0,2017-10-05,50000,01. INDIA,2017-10-05,0,009,LOW,FEE PAYING,VOYAGER


['Account_Number',
 'MISDATE',
 'ATTRITION_FLAG',
 'ACCOUNT_CREATION_DATE',
 'ACTUAL_CURRENT_BALANCE',
 'Credit_Limit',
 'LAST_ACTIVITY_DATE',
 'Last_Statement_Closing_Balance',
 'Last_Statement_Date',
 'Last_Statement_Minimum_Due',
 'Monthly_Salary',
 'Months_On_Book',
 'Next_Statement_Date',
 'Monthly_Salary',
 'Nationality_Group',
 'Next_Statement_Date',
 'No_Of_Due_Stmt',
 'Product',
 'APR_Rate',
 'Product_Fee_Type',
 'Product_Category']

In [ ]:
#Drop garbage first column
Portfolio = Portfolio.drop(Portfolio.index[0])

In [13]:
#Fill in numerical NaN with '0'
Portfolio = Portfolio.reset_index(drop=True)
Portfolio = Portfolio.dropna(subset=['Account_Number'])
Portfolio = Portfolio.dropna(subset=['MISDATE'])
Portfolio['Last_Statement_Closing_Balance'].fillna(0, inplace=True)
Portfolio['ACTUAL_CURRENT_BALANCE'].fillna(0, inplace=True)
Portfolio['Last_Statement_Minimum_Due'].fillna(0, inplace=True)
Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape
Portfolio.head(1)

(55978619, 21)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,MAFF_ACCT_0000000001,2016-11-27,OPEN,2016-11-27,0,10000,NaT,0,2016-11-03,0,...,0,2016-12-03,10000,06. OTHERS,2016-12-03,0,004,LOW,NON FEE,CASHBACK


In [14]:
#Write out to pickle file for later reference
#output = open('Porfolio_Source_v2.pkl', 'wb')
#pickle.dump(Portfolio, output)
#output.close()

#### Filter Portfolio file to just 'open' or 'voluntary

In [15]:
Portfolio['ATTRITION_FLAG'].unique()
Portfolio = Portfolio.loc[(Portfolio['ATTRITION_FLAG']=="OPEN") | (Portfolio['ATTRITION_FLAG']=="VOLUNTARY"), :]
Portfolio['ATTRITION_FLAG'].unique()

array(['OPEN', 'VOLUNTARY'], dtype=object)

array(['OPEN', 'VOLUNTARY'], dtype=object)

In [16]:
# Fill in Numerical NaN with 0
Portfolio['Last_Statement_Closing_Balance'].fillna(0, inplace=True)
Portfolio['ACTUAL_CURRENT_BALANCE'].fillna(0, inplace=True)
Portfolio['Last_Statement_Minimum_Due'].fillna(0, inplace=True)
Portfolio['ATTRITION_FLAG'].unique()
Portfolio['ATTRITION_FLAG'] = Portfolio.ATTRITION_FLAG.astype(str)

Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape

array(['OPEN', 'VOLUNTARY'], dtype=object)

(55978619, 21)

In [17]:
#Format data types 
Portfolio = pd.DataFrame(Portfolio)
Portfolio['MISDATE'] = Portfolio['MISDATE'].astype('datetime64[ns]')
Portfolio['Account_Number'] = Portfolio['Account_Number'].astype('str')
Portfolio.shape

(55978619, 21)

In [24]:

Portfolio.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False], inplace=True)
Portfolio.reset_index(drop=True, inplace=True)

### Take accounts where `Months_On_Book` > 6

In [21]:
Portfolio['Months_On_Book'] = Portfolio['Months_On_Book'].astype(int)
justaccounts = Portfolio.groupby('Account_Number').head(1)
justaccounts.shape

(127999, 21)

In [22]:
newaccounts = justaccounts[justaccounts['Months_On_Book']<7]
newaccounts.head(1)
newaccounts.shape
newaccounts = newaccounts['Account_Number']
type(newaccounts)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,MAFF_ACCT_0000000001,2017-04-25,VOLUNTARY,2016-11-27,0,10000,2017-04-20,0,2017-04-03,0,...,5,2017-05-03,10000,06. OTHERS,2017-05-03,0,004,LOW,NON FEE,CASHBACK


(26011, 21)

pandas.core.series.Series

In [30]:
Portfolio['Account_Number'].nunique()
Portfolio = Portfolio[~Portfolio['Account_Number'].isin(newaccounts)]
Portfolio['Account_Number'].nunique()
Portfolio.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False], inplace=True)
Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape
Portfolio.head(1)
Portfolio.tail(1)

101988

101988

(34813986, 21)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,MAFF_ACCT_0000002125,2017-09-30,OPEN,2012-10-24,12554.14,18000,2017-09-30,17391,2017-09-10,869,...,59,2017-10-10,9000,06. OTHERS,2017-10-10,0,007,LOW,NON FEE,CASHBACK


,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
34813985,MAFF_ACCT_0000267096,2017-03-05,OPEN,2017-02-22,0,15000,NaT,0,2017-02-10,0,...,1,2017-03-10,15000,01. INDIA,2017-03-10,0,009,LOW,FEE PAYING,VOYAGER


### Trim to 365 most recent timesteps for each account number


In [25]:
Portfolio = Portfolio.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
Portfolio = Portfolio.reset_index(drop=True)
Portfolio = Portfolio.groupby('Account_Number').head(365)
Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape
Portfolio.head(1)
Portfolio.tail(1)
#check whether Portfolio data frame has complete #365 day series
Portfolio.shape[0]/365

(37188747, 21)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,MAFF_ACCT_0000000001,2017-04-25,VOLUNTARY,2016-11-27,0,10000,2017-04-20,0,2017-04-03,0,...,5,2017-05-03,10000,06. OTHERS,2017-05-03,0,004,LOW,NON FEE,CASHBACK


,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
37188746,MAFF_ACCT_0000269020,2017-09-30,OPEN,2017-09-30,0,5000,NaT,0,2017-09-05,0,...,0,2017-10-05,50000,01. INDIA,2017-10-05,0,009,LOW,FEE PAYING,VOYAGER


101886.97808219178

### Filter to those accounts with at least one year of account history, or 365 daily timesteps

In [26]:
Portfolio = Portfolio.groupby('Account_Number').filter(lambda x: len(x) > 364)
Portfolio = Portfolio.reset_index(drop=True)
Portfolio.shape
Portfolio.head(2)

(29633255, 21)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Months_On_Book,Next_Statement_Date,Monthly_Salary,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category
0,MAFF_ACCT_0000002125,2017-09-30,OPEN,2012-10-24,12554.14,18000,2017-09-30,17391,2017-09-10,869,...,59,2017-10-10,9000,06. OTHERS,2017-10-10,0,007,LOW,NON FEE,CASHBACK
1,MAFF_ACCT_0000002125,2017-09-29,OPEN,2012-10-24,12454.14,18000,2017-09-29,17391,2017-09-10,869,...,59,2017-10-10,9000,06. OTHERS,2017-10-10,0,007,LOW,NON FEE,CASHBACK


In [28]:
#Confirm that our 365 timesteps are fully populated.
Portfolio.shape[0]/365

81187.0

In [30]:

Portfolio.shape

(29633255, 21)

### Format Dates and Create Date Derived Variables

In [60]:
Portfolio1 = Portfolio
#Format data types 
Portfolio1['Last_Statement_Date'] = Portfolio1['Last_Statement_Date'].astype('datetime64[ns]')
Portfolio1['Next_Statement_Date'] = Portfolio1['Next_Statement_Date'].astype('datetime64[ns]')
Portfolio1['ACCOUNT_CREATION_DATE']=pd.to_datetime(Portfolio1['ACCOUNT_CREATION_DATE'],format='%Y-%m-%d')
Portfolio1['ACCOUNT_CREATION_DATE'] = Portfolio1['ACCOUNT_CREATION_DATE'].astype('datetime64[ns]')
Portfolio1['Account_Number'] = Portfolio1['Account_Number'].astype(str)
Portfolio1['ATTRITION_FLAG'] = Portfolio1['ATTRITION_FLAG'].astype(str)
Portfolio1['LAST_ACTIVITY_DATE']=pd.to_datetime( Portfolio1['LAST_ACTIVITY_DATE'],format='%Y-%m-%d')
Portfolio1['LAST_ACTIVITY_DATE'].fillna(Portfolio1['ACCOUNT_CREATION_DATE'], inplace=True)

In [61]:
Portfolio1['Last_Statement_Minimum_Due'] = Portfolio1['Last_Statement_Minimum_Due'].replace(-np.inf, 0)
Portfolio1['Last_Statement_Minimum_Due'] = Portfolio1['Last_Statement_Minimum_Due'].replace(-np.nan, 0)
Portfolio1['Last_Statement_Minimum_Due'] = Portfolio1['Last_Statement_Minimum_Due'].replace(np.inf, 0)
Portfolio1['Last_Statement_Minimum_Due'] = Portfolio1['Last_Statement_Minimum_Due'].replace(np.nan, 0)

In [62]:
Portfolio1['Monthly_Salary'] = Portfolio1['Monthly_Salary'].astype('int')
Portfolio1['Credit_Limit'] = Portfolio1['Credit_Limit'].astype('int')
Portfolio1['Last_Statement_Closing_Balance'] = Portfolio1['Last_Statement_Closing_Balance'].astype('int')
Portfolio1['Last_Statement_Minimum_Due'] = Portfolio1['Last_Statement_Minimum_Due'].astype('int')

In [63]:
#Days on Book
Portfolio1['ACCOUNT_CREATION_DATE'] = pd.to_datetime(Portfolio1['ACCOUNT_CREATION_DATE'], format='%Y-%m-%d')
Portfolio1['Days_On_Book'] = Portfolio1['MISDATE'] - Portfolio1['ACCOUNT_CREATION_DATE']
Portfolio1['Days_On_Book'] = Portfolio1['Days_On_Book'].dt.days
Portfolio1['Days_On_Book'] = Portfolio1['Days_On_Book'].astype(int)

In [64]:
#Days from Last Statement
Portfolio1['Last_Statement_Date'] = pd.to_datetime(Portfolio1['Last_Statement_Date'], format='%Y-%m-%d')
Portfolio1['Days_From_Statement'] = Portfolio1['MISDATE']-Portfolio1['Last_Statement_Date']
Portfolio1['Days_From_Statement'] = Portfolio1['Days_From_Statement'].dt.days
Portfolio1['Days_From_Statement'] = Portfolio1['Days_From_Statement'].astype(int)

In [65]:
#Days from Last Activity
Portfolio1['LAST_ACTIVITY_DATE'].fillna(Portfolio1['Last_Statement_Date'], inplace=True)
Portfolio1['LAST_ACTIVITY_DATE'].fillna(Portfolio1['MISDATE'], inplace=True)
Portfolio1['LAST_ACTIVITY_DATE']=pd.to_datetime( Portfolio1['LAST_ACTIVITY_DATE'],format='%Y-%m-%d')
Portfolio1['Days_from_Last_Activity'] = Portfolio1['MISDATE'] - Portfolio1['LAST_ACTIVITY_DATE']
Portfolio1['Days_from_Last_Activity'] = Portfolio1['Days_from_Last_Activity'].dt.days
Portfolio1['Days_from_Last_Activity'] = Portfolio1['Days_from_Last_Activity'].replace(-np.inf, 0)
Portfolio1['Days_from_Last_Activity'] = Portfolio1['Days_from_Last_Activity'].replace(-np.nan, 0)
Portfolio1['Days_from_Last_Activity'] = Portfolio1['Days_from_Last_Activity'].astype(int)

In [66]:
Portfolio1.shape

(29633255, 24)

In [67]:
Portfolio1.head(1)

,Account_Number,MISDATE,ATTRITION_FLAG,ACCOUNT_CREATION_DATE,ACTUAL_CURRENT_BALANCE,Credit_Limit,LAST_ACTIVITY_DATE,Last_Statement_Closing_Balance,Last_Statement_Date,Last_Statement_Minimum_Due,...,Nationality_Group,Next_Statement_Date,No_Of_Due_Stmt,Product,APR_Rate,Product_Fee_Type,Product_Category,Days_On_Book,Days_From_Statement,Days_from_Last_Activity
0,MAFF_ACCT_0000002125,2017-09-30,OPEN,2012-10-24,12554.14,18000,2017-09-30,17391,2017-09-10,869,...,06. OTHERS,2017-10-10,0,007,LOW,NON FEE,CASHBACK,1802,20,0


In [68]:
#Reset index, order, check dimensions
Portfolio1 = Portfolio1.sort_values(['Account_Number', 'MISDATE'], ascending=[True, False])
Portfolio1 = Portfolio1.reset_index(drop=True)

In [69]:
#Check counts of unique ids.

In [70]:
uniqueids =Portfolio1.groupby('Account_Number').head(1)
uniqueids = uniqueids.reset_index(drop=True)
uniqueids['ATTRITION_FLAG'].value_counts()

OPEN         74328
VOLUNTARY     6859
Name: ATTRITION_FLAG, dtype: int64

#### Write out interim file

In [71]:
output = open('Portfolio_cleaned.pkl', 'wb')
pickle.dump(Portfolio1, output)
output.close()

In [72]:
del Portfolio

## Import Statement Data from SQL Pull, Pickled and Stored in Blob

In [75]:
mycontainer = "<yourcontainername>"       # fill in the container name
myblobname = "<yourblobname"        # fill in the blob name
mydatafile = "Statements"        # fill in the output file name that you want

In [76]:
dirname = base_dir
blob_service = BlockBlobService(account_name=blob_account_name,
                 account_key=blob_account_key)
blob_service.get_blob_to_path(mycontainer, myblobname, mydatafile)


In [77]:
mydata = pd.read_csv(mydatafile, sep='\t',error_bad_lines=False, index_col=False, dtype='unicode')
print(mydata.shape)

(2130000, 20)


In [78]:
Statements = mydata[['ACCOUNT_NUMBER', 'STATEMENT_DATE', 'BEGINNING_AR', 'CLOSING_AR',
       'TOTAL_MINIMUM_DUE', 'PAYMENT_DUE_DATE', 'POINTS_BALANCE_BEGINNING',
       'POINTS_EARNED', 'POINTS_REDEEMED', 'POINTS_BALANCE_CLOSING',
       'POINTS_ADJUSTED', 'STATEMENT_FROM_DATE', 'STATEMENT_TO_DATE',
       'STATEMENT_BALANCE', 'INSTALLMENTS_REMAINING', 'NEW_INSTALLMENT_FLAG',
       'BT_FLAG', 'LOP_FLAG', 'EPP_FLAG', 'AMF_FLAG']]

del mydata

#### Statements - Reduce and Reformat

In [79]:
Statements.head(1)

,ACCOUNT_NUMBER,STATEMENT_DATE,BEGINNING_AR,CLOSING_AR,TOTAL_MINIMUM_DUE,PAYMENT_DUE_DATE,POINTS_BALANCE_BEGINNING,POINTS_EARNED,POINTS_REDEEMED,POINTS_BALANCE_CLOSING,POINTS_ADJUSTED,STATEMENT_FROM_DATE,STATEMENT_TO_DATE,STATEMENT_BALANCE,INSTALLMENTS_REMAINING,NEW_INSTALLMENT_FLAG,BT_FLAG,LOP_FLAG,EPP_FLAG,AMF_FLAG
0,MAFF_ACCT_0000000001,2017-05-03,1639.27,0.00,0.00,2017-05-28,0.00,0,0,0,0,2017-04-03,2017-05-03,0.00,NaN,N,N,N,N,N


In [80]:
Statements = pd.DataFrame(Statements)
Statements = Statements.dropna(subset=['ACCOUNT_NUMBER'])
Statements = Statements.dropna(subset=['STATEMENT_DATE'])
Statements_Index = Statements.reset_index(drop=True)

Statements_Index = Statements_Index.sort_values(['ACCOUNT_NUMBER', 'STATEMENT_DATE'], ascending=[True, True])
Statements_Index = Statements_Index.reset_index(drop=True)
Statements_Index = Statements_Index[['ACCOUNT_NUMBER','STATEMENT_DATE','STATEMENT_FROM_DATE','STATEMENT_TO_DATE','STATEMENT_BALANCE','POINTS_REDEEMED','POINTS_BALANCE_CLOSING','POINTS_EARNED']]
Statements_Index.columns = ['Account_Number','STATEMENT_DATE','STATEMENT_FROM_DATE','STATEMENT_TO_DATE','STATEMENT_BALANCE','POINTS_REDEEMED','POINTS_BALANCE_CLOSING', 'POINTS_EARNED']
Statements_Index['STATEMENT_DATE'] = Statements_Index['STATEMENT_DATE'].astype('datetime64[ns]')
Statements_Index['STATEMENT_FROM_DATE'] = Statements_Index['STATEMENT_FROM_DATE'].astype('datetime64[ns]')
Statements_Index['STATEMENT_TO_DATE'] = Statements_Index['STATEMENT_TO_DATE'].astype('datetime64[ns]')
Statements_Index['Account_Number'] = Statements_Index['Account_Number'].astype('str')
Statements_Index['STATEMENT_BALANCE'] = Statements_Index['STATEMENT_BALANCE'].astype('float')
Statements_Index['STATEMENT_BALANCE'] = round(Statements_Index['STATEMENT_BALANCE'],0)
Statements_Index['STATEMENT_BALANCE'] = Statements_Index['STATEMENT_BALANCE'].astype('int')
Statements_Index['POINTS_EARNED'] = Statements_Index['POINTS_EARNED'].astype('int')
Statements_Index['POINTS_REDEEMED'] = Statements_Index['POINTS_REDEEMED'].astype('int')
Statements_Index['POINTS_BALANCE_CLOSING'] = Statements_Index['POINTS_BALANCE_CLOSING'].astype('int')

Statements_Index.head(2)
Statements_Index.shape

,Account_Number,STATEMENT_DATE,STATEMENT_FROM_DATE,STATEMENT_TO_DATE,STATEMENT_BALANCE,POINTS_REDEEMED,POINTS_BALANCE_CLOSING,POINTS_EARNED
0,MAFF_ACCT_0000000001,2017-05-03,2017-04-03,2017-05-03,0,0,0,0
1,MAFF_ACCT_0000000003,2017-12-18,2017-11-18,2017-12-18,2049,0,0,0


(2129999, 8)

In [81]:
Statements_Index.dtypes

Account_Number                    object
STATEMENT_DATE            datetime64[ns]
STATEMENT_FROM_DATE       datetime64[ns]
STATEMENT_TO_DATE         datetime64[ns]
STATEMENT_BALANCE                  int32
POINTS_REDEEMED                    int32
POINTS_BALANCE_CLOSING             int32
POINTS_EARNED                      int32
dtype: object

In [82]:
del Statements

### Write out Cleaned Statements DataFrame

In [83]:
Statements_Index.to_csv('Statements_cleaned.csv',sep=',',encoding='utf-8')

In [84]:
del Statements_Index

## Import CRM Data from SQL Pull Stored in Blob

In [87]:
mycontainer = "<YourContainerName>"       # fill in the container name
myblobname = "<YourBlobName>"        # fill in the blob name
mydatafile = "<YourDataName>"        # fill in the output file name

In [88]:
blob_service = BlockBlobService(account_name=blob_account_name,
                 account_key=blob_account_key)
blob_service.get_blob_to_path(mycontainer, myblobname, mydatafile)

CRM = pd.read_csv(mydatafile, sep='\t',error_bad_lines=False, index_col=False, dtype='unicode')
print(CRM.shape)

(2176494, 9)


In [90]:
CRM.dtypes

Incident_number      object
Account_Number       object
Incident_ID          object
Incident_Category    object
Sub_Category         object
Incident_Type        object
Created_On           object
Updated_On           object
Closed_date          object
dtype: object

#### Reformat Variables

In [91]:
CRM = CRM.sort_values(['Account_Number'], ascending=[ True])
CRM = CRM.reset_index(drop=True)
CRM = CRM[['Account_Number','Incident_Category','Incident_Type','Created_On']]
CRM['Incident_Category'] = CRM['Incident_Category'].astype('str')
CRM['Incident_Type'] = CRM['Incident_Type'].astype('str')
CRM['Created_On'] = CRM['Created_On'].astype('datetime64[ns]')
CRM = CRM.dropna(subset=['Account_Number'])
CRM = CRM.dropna(subset=['Created_On'])
CRM = CRM.reset_index(drop=True)

CRM.head(1)
CRM.shape
CRM.dtypes

,Account_Number,Incident_Category,Incident_Type,Created_On
0,MAFF_ACCT_0000000002,Voyager Booking,Voyager,2017-08-02


(2176209, 4)

Account_Number               object
Incident_Category            object
Incident_Type                object
Created_On           datetime64[ns]
dtype: object

#### Remove Very infrequent incident Types

In [92]:
print('number of unique account numbers',len(CRM.Account_Number.unique()))
print('number of unique incident categorices',(len(CRM.Incident_Category.unique())))
#CRM['Incident_Type'].value_counts()

number of unique account numbers 146296
number of unique incident categorices 359


In [93]:
CRM.shape
#remove incidents with less than 100 occurences
threshold = 99
counts = CRM['Incident_Type'].value_counts()

CRM2 = CRM.loc[CRM['Incident_Type'].isin(counts[counts > threshold].index), :]
CRM2 = CRM2.loc[(CRM2['Incident_Type']!="Inbound SMS")]
CRM2 = CRM2.reset_index(drop=True)
print('number of unique account numbers',len(CRM2.Account_Number.unique()))
print('number of unique incident categorices',(len(CRM2.Incident_Category.unique())))
CRM2.shape

(2176209, 4)

number of unique account numbers 143317
number of unique incident categorices 338


(2102570, 4)

In [94]:
CRM2.to_csv('CRM_cleaned.csv',sep=',',encoding='utf-8')

## Import Aggregated Transaction Data from SQL Pull Stored in Blob

In [95]:
mycontainer = "<yourcontainer>"       # fill in the container name
myblobname = "<yourblobname>"        # fill in the blob name
mydatafile = "TransactionsDailyAgg"        # fill in the output file name

In [96]:
blob_service = BlockBlobService(account_name=blob_account_name,
                 account_key=blob_account_key)
blob_service.get_blob_to_path(mycontainer, myblobname, mydatafile)

Transactions_DailyAgg = pd.read_csv(mydatafile, sep='\t',error_bad_lines=False, index_col=False, dtype='unicode')
print(Transactions_DailyAgg.shape)

(15097872, 16)


In [97]:

Transactions_DailyAgg.dtypes

Account_Number                      object
Purchase_Date                       object
Billing_Amount_Sum_Payment          object
Billing_Amount_Sum_Spend_ESS        object
Billing_Amount_Sum_Spend_NON_ESS    object
Billing_Amount_Spend_CASH           object
Billing_Amount_Violation_Fee        object
Transaction_Count_Payment           object
Tranasaction_Count_Spend_ESS        object
Tranasaction_Count_Spend_NON_ESS    object
Tranasaction_Count_Spend_CASH       object
Transaction_Count_Foreclosure       object
Tranasaction_Count_BT               object
Tranasaction_Count_LOP              object
Tranasaction_Count_EPP              object
Tranasaction_Count_Violation_Fee    object
dtype: object

In [98]:
Transactions_DailyAgg.rename(columns = {
      'sum' : 'Billing_Amount_Sum',
      'count' : 'Transaction_Count'},inplace = True)

In [99]:
output = open('Transactions_cleaned.pkl', 'wb')
pickle.dump(Transactions_DailyAgg, output)
output.close()

In [100]:
#Transactions_DailyAgg.to_csv('Transactions_DailyAgg_cleaned.csv',sep=',',encoding='utf-8')

### End Part 1 - Cleaning each source table